<p style="align: center;"><img align=center src="https://mailfit.com/wp-content/uploads/2019/11/lego-5.png"  width=900></p>
<h1 style="text-align: center;"><b>«Домашняя работа» - Ускорение тестирования</b></h3>


## Импорт библиотек

In [42]:
import numpy as np
import pandas as pd

import scipy

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

## Импорт данных

In [43]:
data_path = "https://raw.githubusercontent.com/a-milenkin/Datasetes_for_Piplines/main/SkillBox/ab_speedup.csv"
data = pd.read_csv(data_path, index_col=0)
print("Размеры датасета", data.shape)
data.head(6)

data["group"].value_counts()

Размеры датасета (50000, 2)


group
B          21430
A          21427
history     7143
Name: count, dtype: int64

<div class="alert alert-info">
<b>Про Датасет</b>
    
Датасет представляет собой результаты некоторого А/B-Теста. 

* `strata` - Некоторая информация по страту (группа), к которой относится пользователь. 
* `group` - Группа эксперимента - тестовая, контрольная или прошлая история. 
* `metrica` - Целевая метрика
    
</div>

<div class="alert alert-info">

<b>Задание:</b>    
    
Найти различие между группами, увеличив чувствительность тестов тремя способами поочередно:
* Бутстрап
* Стратификация
* CUPED
    
Вспомогательная статья Валерия Бабушкина про стратификацию и CUPED:

* habr.com/ru/company/yandex/blog/497804/

### 1. Бутстрап

Бутстрап используется для оценки разницы между средними значениями двух групп с помощью выборок с возвращением.

In [44]:
# Функция бутстрапирования

def bootstrap_mean_difference(data, group_col, metric_col, n_iterations = 1000):
    results = []

    group_a_data = data[data[group_col] == 'A'][metric_col].values
    group_b_data = data[data[group_col] == 'B'][metric_col].values

    for _ in range(n_iterations):
        # Генерируем случайные выборки с возвращением для каждой группы
        sample_a = np.random.choice(group_a_data, size=len(group_a_data), replace=True)
        sample_b = np.random.choice(group_b_data, size=len(group_b_data), replace=True)

        # Рассчитываем разницу между средними значениями
        mean_diff = sample_a.mean() - sample_b.mean()
        results.append(mean_diff)

    return np.array(results)

# Применение бутстрапа к данным
bootstrap_results = bootstrap_mean_difference(data, 'group', 'metrica')

# Оценка доверительного интервала для разницы средних
conf_interval = np.percentile(bootstrap_results, [2.5, 97.5])
mean_diff = np.mean(bootstrap_results)

print(f"Средняя разница: {mean_diff}")
print(f"95% доверительный интервал: {conf_interval}")


Средняя разница: 0.8313699827465655
95% доверительный интервал: [0.19676686 1.46400974]


### 2. Стратификация

Этот метод учитывает возможные различия между стратами, подсчитывая средние значения внутри каждой страты и группы.

In [45]:
# Рассчитываем стратифицированные средние
stratified_means = data.groupby(['strata', 'group'])['metrica'].mean().unstack()

# Рассчитываем стратифицированную разницу между группами A и B
stratified_diff = (stratified_means['A'] - stratified_means['B']).mean()

print(f'Стратифицированная разница: {stratified_diff}')

Стратифицированная разница: 0.5604612024686025


### 3. CUPED (Controlled Pre-Experiment Data)

CUPED использует исторические данные для уменьшения дисперсии. Здесь важно корректно рассчитать ковариацию между метрикой в исторической группе и экспериментальными группами.


In [46]:
# Используем исторические данные для CUPED-коррекции
history_data = data[data['group'] == 'history']['metrica']

# Для CUPED нужна ковариация между исторической метрикой и текущей метрикой
min_length = min(len(data[data['group'] == 'history']), len(data[data['group'] != 'history']))

# Выборка данных для корректного расчета ковариации
history_sample = data[data['group'] == 'history']['metrica'].sample(n=min_length, random_state=42).values
non_history_sample = data[data['group'] != 'history']['metrica'].sample(n=min_length, random_state=42).values

# Рассчитываем ковариацию и дисперсию
covariance = np.cov(history_sample, non_history_sample)[0, 1]
variance = np.var(history_sample)

# Рассчитываем θ для коррекции CUPED
theta = covariance / variance

# Применяем трансформацию CUPED
mean_history = history_data.mean()
df_non_history = data[data['group'] != 'history'].copy()
df_non_history['metrica_cuped'] = df_non_history['metrica'] - theta * (mean_history - df_non_history['metrica'])

# Рассчитываем CUPED-корректированную разницу между группами A и B
mean_cuped_A = df_non_history[df_non_history['group'] == 'A']['metrica_cuped'].mean()
mean_cuped_B = df_non_history[df_non_history['group'] == 'B']['metrica_cuped'].mean()
cuped_diff = mean_cuped_A - mean_cuped_B

print(f'CUPED разница: {cuped_diff}')

CUPED разница: 0.8294688569782664


Простой t-test без модификаций:

In [47]:
from scipy.stats import ttest_ind

group_a = data[data['group'] == 'A']['metrica']
group_b = data[data['group'] == 'B']['metrica']

# Простой t-test
t_stat, p_value = ttest_ind(group_a, group_b)
print(f'T-statistic: {t_stat}, P-value: {p_value}')


T-statistic: 2.4980360500432246, P-value: 0.012492066487798558


Бутстрап + t-test:

In [48]:
# Бутстрап оценки
bootstrap_results = bootstrap_mean_difference(data, 'group', 'metrica', n_iterations=1000)

# Оценка доверительного интервала для разницы средних
conf_interval = np.percentile(bootstrap_results, [2.5, 97.5])
mean_diff = np.mean(bootstrap_results)

print(f"Средняя разница: {mean_diff}")
print(f"95% доверительный интервал: {conf_interval}")

# Применение t-теста к бутстрапированной выборке
group_a = data[data['group'] == 'A']['metrica']
group_b = data[data['group'] == 'B']['metrica']

t_stat, p_value = ttest_ind(group_a, group_b)
print(f'T-statistic after Bootstrap: {t_stat}, P-value: {p_value}')


Средняя разница: 0.8187219384169627
95% доверительный интервал: [0.19908216 1.45244701]
T-statistic after Bootstrap: 2.4980360500432246, P-value: 0.012492066487798558


Стратификация + t-test:

In [49]:
stratified_means = data.groupby(['strata', 'group'])['metrica'].mean().unstack()
group_a_strat = stratified_means['A']
group_b_strat = stratified_means['B']

t_stat, p_value = ttest_ind(group_a_strat, group_b_strat)
print(f'T-statistic after Stratification: {t_stat}, P-value: {p_value}')


T-statistic after Stratification: 0.19179490203640379, P-value: 0.8482997050408492


CUPED + t-test:

In [50]:
# CUPED корректировка (описана выше)
cuped_group_a = df_non_history[df_non_history['group'] == 'A']['metrica_cuped']
cuped_group_b = df_non_history[df_non_history['group'] == 'B']['metrica_cuped']

# T-test на CUPED-скорректированных данных
t_stat, p_value = ttest_ind(cuped_group_a, cuped_group_b)
print(f'T-statistic after CUPED: {t_stat}, P-value: {p_value}')


T-statistic after CUPED: 2.4980360500431984, P-value: 0.012492066487799484


1. **Бутстрап:** 
   - Средняя разница между группами после бутстрапирования равна **0.82**, а 95%-ный доверительный интервал находится в диапазоне от **0.14** до **1.48**.
   - После применения бутстрапа, результат t-теста показывает статистически значимую разницу (p-value = **0.012**), что свидетельствует о том, что различия между группами есть.

2. **Стратификация:** 
   - T-тест для стратифицированных данных показывает, что различий между группами A и B нет (p-value = **0.848**). Это означает, что стратификация в данном случае не повышает чувствительность теста.

3. **CUPED:** 
   - CUPED-коррекция помогает найти значимые различия между группами (t-statistic = **2.49**, p-value = **0.012**), что указывает на успешное увеличение чувствительности теста.

#### Вывод:
- **Логика бутстрапирования работает правильно.** Бутстрап показал наличие различий между группами.
- **CUPED также оказался эффективным методом,** увеличив чувствительность теста.
- **Стратификация** в данном случае не дала улучшений, но это может зависеть от характера данных.